In [1]:
import basicModel as bm
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
#股票信息数据
infodf = pd.read_feather('./database/infoall.txt')
newsdf = pd.read_feather('./database/all_news_withVEC_BasicCN.txt').set_index('index')

In [3]:
#模型初始化
fb = bm.basicmodel()
fb.initialize()

In [7]:
fb.close / fb.close.shift(1) -1

,000001.SZ,000002.SZ,000003.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,688582.SH,688620.SH,688623.SH,688629.SH,688631.SH,832175.BJ,832651.BJ,833455.BJ,836208.BJ,836221.BJ
time,,,,,,,,,,,,,,,,,,,,,
2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-02,-0.009233,-0.009508,NaN,0.045596,0.010695,-0.028634,-0.049865,0.007491,0.028800,0.101408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-06,0.015771,-0.023715,NaN,0.028741,0.000000,-0.009070,0.001418,-0.022305,0.060653,0.099744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-07,-0.000706,0.008097,NaN,-0.035645,-0.005291,0.020595,0.008499,-0.011407,-0.017595,0.016279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-08,0.000000,0.006311,NaN,-0.010989,0.000000,-0.029148,-0.004213,-0.015385,0.032090,-0.032037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,-0.014978,-0.013630,NaN,-0.028934,0.017241,-0.015982,0.051282,-0.008621,-0.014050,0.004831,...,NaN,NaN,0.003772,NaN,NaN,0.000000,0.000000,-0.102725,0.000000,-0.071186
2023-06-27,0.008945,0.042182,NaN,0.000000,0.016949,0.037123,0.048780,0.017391,0.011735,0.043269,...,NaN,NaN,-0.011057,NaN,NaN,0.000000,-0.169286,0.021028,0.000000,-0.060827
2023-06-28,0.001773,-0.007676,NaN,0.100291,0.033333,-0.026846,0.049096,-0.004274,-0.016570,0.000000,...,NaN,-0.100770,-0.030612,-0.096495,NaN,0.000000,-0.079966,-0.091533,0.000000,-0.034974


In [5]:
#获取交易日
close = pd.read_feather('./database/BasicFactor_Close.txt').set_index('time')
close.index = pd.to_datetime(close.index.astype(str))
close = close['2022-07-01':]
tradeday = close.index
#交易日股票的交易状态
tradestatus = close.mask(close>0,1)

In [40]:
fb.initialize_stockinfo()
day_high_simlist = []
for tradedate in tqdm(tradeday):
    daynewsdf = newsdf[newsdf['交易日']==tradedate.date()]
    for news in daynewsdf.iterrows():
        day_high_simlist.append(fb.corr_max_stock(news[1]['full_sen_vec'],tradedate,5,info_name='shortname'))
newsdf['high_corr_stock'] = day_high_simlist

100%|██████████| 243/243 [05:07<00:00,  1.26s/it]


In [43]:
reslist = []
for news in newsdf.iterrows():
    for simpair in news[1]['high_corr_stock']:
        if simpair[1] > 0.85:
            print(news[1][1:9])
            print(infodf.loc[simpair[0]]['name'])

上榜时间        2022-07-07 00:00:12
热搜              癌症妈妈拼命生下2斤多早产宝宝
最高排名                          2
在榜时长（分钟）                  13时1分
热度                       190901
交易日                  2022-07-07
时间                      preopen
全文                        '打歌🎤'
Name: 2476, dtype: object
深圳市三利谱光电科技股份有限公司
上榜时间        2022-07-07 00:00:12
热搜              癌症妈妈拼命生下2斤多早产宝宝
最高排名                          2
在榜时长（分钟）                  13时1分
热度                       190901
交易日                  2022-07-07
时间                      preopen
全文                        '打歌🎤'
Name: 2476, dtype: object
珠海英搏尔电气股份有限公司
上榜时间        2022-07-07 00:00:12
热搜              癌症妈妈拼命生下2斤多早产宝宝
最高排名                          2
在榜时长（分钟）                  13时1分
热度                       190901
交易日                  2022-07-07
时间                      preopen
全文                        '打歌🎤'
Name: 2476, dtype: object
牧高笛户外用品股份有限公司
上榜时间        2022-07-07 00:00:12
热搜              癌症妈妈拼命生下2斤多早产宝宝
最高排名                          2
在榜时长（分钟）     